# 使用 Kaggle 網站的 OnlineRetail 資料集，進行顧客生命週期價值（CLV）計算
三重點：<br>
一、問題定義：界定問題範圍、主要分析目的、欲尋找答案<br>
二、資料清整：根據所界定問題，有邏輯的清整資料<br>
三、計算 CLV：根據需要，求得所需的 CLV<br>

### 一、問題定義
1. 主要目的是練習計算 CLV。算法有：簡單的個別客戶 x 群體流失率價值計算，也有整群客戶以群體模型預測的方式計算，或個別客戶個別預測（？）。此次使用第一種算法。<br>
2. 為求取 CLV，須至少有客戶 ID、每次客戶交易金額、每次交易日期三類資料。<br>
3. 透過在網路上尋找，找到 Kaggle 上面的 [Online Retail 資料集](https://www.kaggle.com/vijayuv/onlineretail)，包含可產出前述三類資料之資訊。此資料集為英國某零售公司之交易資料，除所需資料外，尚包含發票號碼、存貨號碼、交易內容描述及交易國家等四資料，資料期間為 8 個月，從 2010/10/01 至 2011/12/09，共 541,908 筆資料。

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df_onliret = pd.read_csv('OnlineRetail.csv', encoding= 'unicode_escape', thousands = ',', parse_dates = ['InvoiceDate'])
df_onliret.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


檢視資料表頭，確認是 8 欄位資料沒錯。

### 二、資料清整

In [4]:
#檢視資料各屬性，看是否需調整或刪減；並可於刪減資料後、運算資料前，再次處理資料型態，因此時可能因為資料刪減，而使得資料內容更好被辨認是否為所需資料
df_onliret.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


CustomerID 性質應該是屬於不可運算的 string，故進行調整：

In [5]:
df_onliret.CustomerID = df_onliret.CustomerID.astype(str)
df_onliret.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   541909 non-null  object        
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB


In [6]:
#留下需要的欄位即可：#0,3,4,5,6，可使用 7 來檢視是否需進一步鎖定特定國家的客戶群進行計算（假設流失狀況會因國家而不同）
df_onliret.Country.describe()

count             541909
unique                38
top       United Kingdom
freq              495478
Name: Country, dtype: object

發現顧客來自於 38 個國家，進一步檢視資料分佈，看是否選擇特定國家就好？

In [7]:
b = df_onliret.Country.value_counts()
b

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

求得第一名國家為 UK，進一步了解占比，已更確認資料集中狀況：

In [8]:
for i in b:
    c = i / 541909
    print(f'{c:.2f}')

0.91
0.02
0.02
0.02
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00


UK 占比 91%，在假設不同國家可能有不同購物習性，而有不同流失率的狀況下，決定此次僅使用最大宗的 UK 資料，故刪除其他資料

In [9]:
df_onliret = df_onliret[df_onliret.Country == 'United Kingdom']
df_onliret

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,2011-12-09 12:31:00,1.95,15804.0,United Kingdom
541890,581586,22061,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom
541891,581586,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,2011-12-09 12:49:00,1.25,13113.0,United Kingdom
541892,581586,21217,RED RETROSPOT ROUND CAKE TINS,24,2011-12-09 12:49:00,8.95,13113.0,United Kingdom


驗證：刪除後列數等於前面計算出來的 UK 資料列數，故並未出現前端資料輸入字串多帶空格等問題

In [10]:
#檢視其他欄位是否存在異常資料
df_onliret.describe().round()

,Quantity,UnitPrice
count,495478.0,495478.0
mean,9.0,5.0
std,228.0,99.0
min,-80995.0,-11062.0
25%,1.0,1.0
50%,3.0,2.0
75%,10.0,4.0
max,80995.0,38970.0


發現 Quantity 和 UniPrice 有負值，調閱資料了解詳情，是否可能為退貨等因素？

In [11]:
#依貨物負值數量大小看前 20 名，是否有異常
df_onliret[df_onliret['Quantity'] < 0]['Quantity'].value_counts().sort_index().head(20)  

-80995    1
-74215    1
-9600     2
-9360     1
-9058     1
-5368     1
-4830     1
-3667     1
-3167     1
-3114     1
-3100     1
-3000     1
-2880     1
-2834     1
-2618     1
-2600     1
-2472     1
-2376     1
-2003     1
-2000     1
Name: Quantity, dtype: int64

看似無異常

In [12]:
#依被退次數最多之貨物數量多寡檢視前 20 名，看是否有異常
df_onliret[df_onliret['Quantity'] < 0]['Quantity'].value_counts().sort_values(ascending = False).head(20)

-1      3652
-2      1177
-3       514
-12      442
-4       427
-6       415
-24      215
-5       208
-10      166
-8       154
-7        92
-48       79
-9        73
-36       60
-20       59
-11       57
-100      57
-30       49
-16       47
-18       47
Name: Quantity, dtype: int64

看似無異常

In [13]:
#依樣檢視 UnitPrice
df_onliret[df_onliret['UnitPrice'] <= 0]['UnitPrice'].value_counts().sort_index(ascending = True).head(20)  

-11062.06       2
 0.00        2497
Name: UnitPrice, dtype: int64

發現有負值和 0 二類資訊，預計檢視負值之資料列，而 0 之資料列則因數量眾多，判斷為正常狀況，擬不予特別檢視

In [14]:
#檢視 UnitPrice 負值所在整列資訊
df_onliret[df_onliret['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,nan,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,nan,United Kingdom


發現是為了調整壞帳所入之交易紀錄，但可能為往年資料，所以未直接抵銷在資料集中的特定客戶交易資料，故也無對應 CumstomerID。判斷與欲使用資料無關連，故予以刪除：

In [15]:
df_onliret = df_onliret[df_onliret['UnitPrice'] >= 0]
df_onliret

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,2011-12-09 12:31:00,1.95,15804.0,United Kingdom
541890,581586,22061,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom
541891,581586,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,2011-12-09 12:49:00,1.25,13113.0,United Kingdom
541892,581586,21217,RED RETROSPOT ROUND CAKE TINS,24,2011-12-09 12:49:00,8.95,13113.0,United Kingdom


### 3. 計算 CLV

In [16]:
#首先因資料及缺乏每次交易金額合計欄位，故予以計算
c = df_onliret['UnitPrice'] * df_onliret['Quantity']
df_onliret = df_onliret.assign( Total_Purchase = c)
df_onliret

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Purchase
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,2011-12-09 12:31:00,1.95,15804.0,United Kingdom,23.40
541890,581586,22061,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom,23.60
541891,581586,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,2011-12-09 12:49:00,1.25,13113.0,United Kingdom,30.00
541892,581586,21217,RED RETROSPOT ROUND CAKE TINS,24,2011-12-09 12:49:00,8.95,13113.0,United Kingdom,214.80


In [17]:
#依照客戶 ID 做 groupby
df_gby = df_onliret.groupby('CustomerID').agg({'InvoiceDate': lambda date: (max(date) - min(date)).days,
                                               'InvoiceNo': lambda num: len(num),
                                               'Quantity': lambda quant: sum(quant),
                                               'Total_Purchase': lambda Total_Purchase: sum(Total_Purchase)})
df_gby 

,InvoiceDate,InvoiceNo,Quantity,Total_Purchase
CustomerID,,,,
12346.0,0,2,0,0.00
12747.0,366,103,1275,4196.01
12748.0,372,4642,24210,29072.10
12749.0,209,231,1422,3868.20
12820.0,323,59,722,942.34
...,...,...,...,...
18281.0,0,7,54,80.82
18282.0,118,13,98,176.60
18283.0,333,756,1397,2094.88


In [18]:
#因 Groupby 後有針對欄位重新計算，故根據各欄性質重新命名，以幫助解讀
df_gby.columns = ['Num_Days', 'Num_Transactions', 'Num_Units', 'Total_Purchase']
df_gby.head()

,Num_Days,Num_Transactions,Num_Units,Total_Purchase
CustomerID,,,,
12346.0,0,2,0,0.00
12747.0,366,103,1275,4196.01
12748.0,372,4642,24210,29072.10
12749.0,209,231,1422,3868.20
12820.0,323,59,722,942.34


In [19]:
#刪除客戶 ID 無資料的列
df_gby = df_gby[0:-1]
df_gby

,Num_Days,Num_Transactions,Num_Units,Total_Purchase
CustomerID,,,,
12346.0,0,2,0,0.00
12747.0,366,103,1275,4196.01
12748.0,372,4642,24210,29072.10
12749.0,209,231,1422,3868.20
12820.0,323,59,722,942.34
...,...,...,...,...
18280.0,0,10,45,180.60
18281.0,0,7,54,80.82
18282.0,118,13,98,176.60


In [20]:
#計算客戶流失率 Churn Rate
churn_rate = len(df_gby[df_gby['Num_Transactions'] == 1]) / len(df_gby)
churn_rate

0.019240506329113925

In [21]:
#假設商品平均毛利率為 5%，故計算 CLV 為
CLV = df_gby['Total_Purchase'] * 0.05 / churn_rate
CLV = CLV.round(2)
CLV

CustomerID
12346.0        0.00
12747.0    10904.10
12748.0    75549.21
12749.0    10052.23
12820.0     2448.84
             ...   
18280.0      469.32
18281.0      210.03
18282.0      458.93
18283.0     5443.93
18287.0     4774.51
Name: Total_Purchase, Length: 3950, dtype: float64

In [22]:
#放入資料表中
df_CLV = df_gby.assign( CLV = CLV)
df_CLV.sort_values(by = 'CLV', ascending = False).head(10)  #檢視前 10 大客戶 CLV

,Num_Days,Num_Transactions,Num_Units,Total_Purchase,CLV
CustomerID,,,,,
18102.0,366,433,64122,256438.49,666402.65
17450.0,359,351,69029,187482.17,487206.95
17511.0,370,1076,63012,88125.38,229010.03
16684.0,353,281,49390,65892.08,171232.71
13694.0,369,585,61803,62653.10,162815.62
15311.0,373,2491,37720,59419.34,154412.10
13089.0,366,1857,30787,57385.88,149127.78
14096.0,97,5128,16335,57120.91,148439.21
15061.0,368,410,28590,54228.74,140923.37
